<a href="https://colab.research.google.com/github/jc533/Song_data_game/blob/attackofthetitan-patch-1/datagame.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# import

In [1]:
import pandas as pd
import numpy as np
import os
# from google.colab import drive

!pip install missingno
import missingno as msno

from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV, StratifiedKFold
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler
# drive.mount('/content/drive')
!pip3 install lightgbm
import lightgbm as lgbm

# data viz
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
import plotly.express as px
%matplotlib inline

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [75]:
import warnings
warnings.filterwarnings("ignore")

# read file

In [110]:
path = 'C:/Users/ethan/Downloads/datagame-2023'
train = pd.read_parquet(f'{path}/label_train_source.parquet')
test = pd.read_parquet(f'{path}/label_test_source.parquet')
target = pd.read_parquet(f'{path}/label_train_target.parquet')
composer = pd.read_parquet(f'{path}/meta_song_composer.parquet')
genre = pd.read_parquet(f'{path}/meta_song_genre.parquet')
lyricist = pd.read_parquet(f'{path}/meta_song_lyricist.parquet')
producer = pd.read_parquet(f'{path}/meta_song_producer.parquet')
titletext = pd.read_parquet(f'{path}/meta_song_titletext.parquet')
song = pd.read_parquet(f'{path}/meta_song.parquet')


# merge data


In [111]:
#song_data = pd.concat([train, target])
#song_data = song_data.sort_values(by=['session_id', 'listening_order'])
#song_data = pd.merge(train, target, on='session_id', how='inner')
song_data = pd.merge(train, song, on='song_id', how='left')
song_data = pd.merge(song_data, composer, on='song_id', how='left')
song_data = pd.merge(song_data, genre, on='song_id', how='left')
song_data = pd.merge(song_data, lyricist, on='song_id', how='left')
song_data = pd.merge(song_data, producer, on='song_id', how='left')
song_data = pd.merge(song_data, titletext, on='song_id', how='left')
song_vali = pd.merge(target, song, on='song_id', how='left')
song_vali = pd.merge(song_vali, composer, on='song_id', how='left')
song_vali = pd.merge(song_vali, genre, on='song_id', how='left')
song_vali = pd.merge(song_vali, lyricist, on='song_id', how='left')
song_vali = pd.merge(song_vali, producer, on='song_id', how='left')
song_vali = pd.merge(song_vali, titletext, on='song_id', how='left')


In [115]:
song_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11445180 entries, 0 to 52895870
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   session_id       int64  
 1   song_id          object 
 2   unix_played_at   int64  
 3   play_status      int64  
 4   login_type       int64  
 5   listening_order  int64  
 6   artist_id        float64
 7   song_length      float64
 8   album_id         float64
 9   language_id      float64
 10  album_month      object 
 11  composer_id      object 
 12  genre_id         object 
 13  lyricist_id      object 
 14  producer_id      object 
 15  title_text_id    object 
dtypes: float64(4), int64(5), object(7)
memory usage: 1.4+ GB


In [116]:
song_vali.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2861295 entries, 0 to 13159147
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   session_id       int64  
 1   song_id          object 
 2   unix_played_at   int64  
 3   play_status      int64  
 4   login_type       int64  
 5   listening_order  int64  
 6   artist_id        float64
 7   song_length      float64
 8   album_id         float64
 9   language_id      float64
 10  album_month      object 
 11  composer_id      object 
 12  genre_id         object 
 13  lyricist_id      object 
 14  producer_id      object 
 15  title_text_id    object 
dtypes: float64(4), int64(5), object(7)
memory usage: 371.1+ MB


In [117]:
song_data.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)
song_vali.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)

In [118]:
for i in song_data.select_dtypes(include=['object']).columns:
  song_data[i][song_data[i].isnull()] = 'unknown'
song_data = song_data.fillna(value=0)

for i in song_vali.select_dtypes(include=['object']).columns:
  song_vali[i][song_vali[i].isnull()] = 'unknown'
song_vali = song_vali.fillna(value=0)

label_encoder = LabelEncoder()
one_hot = OneHotEncoder()

for i in song_data.columns:
  song_data[i] = label_encoder.fit_transform(song_data[i])
for i in song_vali.columns:
  song_vali[i] = label_encoder.fit_transform(song_vali[i])
song_data.reset_index(drop=True)
song_vali.reset_index(drop=True)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
0,246,161432,226329,2,3,0,5178,232,57005,1,666,42322,25,27743,22667,754
1,246,17592,226331,2,3,1,5178,232,0,0,712,36282,38,23754,712,75
2,246,106096,226392,2,3,2,2225,278,20032,1,615,59876,30,39204,15615,754
3,246,201297,226438,1,3,3,5178,255,0,0,712,63764,38,41766,22667,75
4,246,25929,226441,2,3,4,5178,237,0,0,712,1921,38,27743,712,174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861290,571663,117218,2177021,1,2,0,50531,116,123183,8,698,63764,17,41766,22667,798
2861291,571663,159104,2177026,1,2,1,50531,68,123183,8,698,63764,16,41766,22667,798
2861292,571663,83258,2177035,1,2,2,50531,63,123183,8,698,63764,16,41766,22667,798
2861293,571663,101811,2177036,1,2,3,50531,63,123183,8,698,63764,17,41766,22667,798


In [119]:
song_data.head()

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
0,600,244261,285531,2,2,0,40939,262,85508,1,704,54037,29,34728,34243,792
2,600,10388,285763,2,2,1,68329,110,0,0,760,28709,37,18434,28060,1504
3,600,10388,285765,1,2,2,68329,110,0,0,760,28709,37,18434,28060,1504
4,600,10388,285766,1,2,3,68329,110,0,0,760,28709,37,18434,28060,1504
5,600,353189,285797,2,2,4,78606,292,0,0,760,5285,37,3438,34243,338


In [122]:
training = song_data.groupby('session_id')
targeting = song_vali.groupby('session_id')

#training = song_data.merge()

In [123]:
training.get_group(751)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
30213348,751,443804,1363031,1,3,0,0,0,0,0,760,102271,37,65290,34243,2055
30213349,751,25999,1363195,1,3,1,0,0,0,0,760,102271,37,65290,34243,2055
30213350,751,540067,1363388,1,3,2,54661,227,178973,3,736,102271,10,65290,34243,196
30213352,751,429284,1363482,2,3,3,15833,236,0,0,760,102271,37,65290,34243,1504
30213353,751,572931,1363531,2,3,4,15291,305,19539,3,639,100449,24,11434,34243,1504
30213355,751,429284,1363561,1,3,5,15833,236,0,0,760,102271,37,65290,34243,1504
30213356,751,220918,1363752,1,3,6,0,0,194103,3,621,102271,24,65290,34243,2055
30213358,751,73010,1363777,1,3,7,4760,319,112068,8,633,60478,16,9491,34243,576
30213360,751,607782,1363991,1,3,8,4760,218,112068,8,633,60478,8,55794,34243,576
30213362,751,240876,1364015,2,3,9,4760,312,112021,8,665,60478,8,9491,34243,576


In [124]:
targeting.get_group(751)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
8659344,751,274093,568179,1,3,0,16443,260,0,0,712,22434,38,5819,22667,146
8659345,751,311304,568230,1,3,1,19089,301,74036,3,573,3441,10,41766,22667,948
8659347,751,311304,568239,1,3,2,19089,301,74036,3,573,3441,10,41766,22667,948
8659349,751,327888,568255,1,3,3,84,227,0,0,712,63764,38,41766,22667,1093
8659350,751,318261,568258,1,3,4,84,301,0,0,712,63764,38,41766,22667,1093


# Preprocess


In [99]:
for i in test.columns:
  test[i] = label_encoder.fit_transform(test[i])
test.head(50)


,session_id,song_id,unix_played_at,play_status,login_type,listening_order
0,118,89565,299096,2,2,0
1,118,194933,299203,2,2,1
2,118,109275,299237,2,2,2
3,118,98197,299238,2,2,3
4,118,250870,299240,2,2,4
5,118,154531,299241,2,2,5
6,118,24559,299242,2,2,6
7,118,183641,299243,2,2,7
8,118,49060,299245,2,2,8
9,118,243906,299246,2,2,9


In [103]:
training.get_group(1)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
8886521,1,9099,1544057,2,2,0,0,0,73963,1,698,102271,24,65290,34243,2055
8886523,1,164152,1544197,2,2,1,0,0,111424,1,716,102271,29,65290,34243,2055
8886525,1,618208,1544229,2,2,2,0,0,188294,1,738,102271,29,65290,34243,2055
8886527,1,71629,1544245,2,2,3,79584,253,185280,1,737,36069,24,29136,14984,1393
8886531,1,574233,1544251,1,2,4,0,0,95719,1,709,102271,24,65290,34243,2055
8886533,1,219549,1544252,2,2,5,7660,288,0,0,760,26217,37,52784,23411,1393
8886535,1,372056,1544256,2,2,6,7155,258,0,0,760,102271,37,65290,34243,98
8886536,1,90818,1544258,2,2,7,42697,220,110407,1,716,83954,29,53756,14984,1393
8886538,1,428781,1544263,2,2,8,786,267,142388,1,726,72743,24,7440,14984,1393
8886542,1,499369,1544268,2,2,9,5679,245,92359,1,708,39412,24,25629,10292,1032


In [104]:
targeting.get_group(1)

,session_id,song_id,unix_played_at,play_status,login_type,listening_order,artist_id,song_length,album_id,language_id,album_month,composer_id,genre_id,lyricist_id,producer_id,title_text_id
4328443,1,282056,641313,2,2,0,14733,255,16187,1,607,13852,25,36300,15615,754
4328445,1,178483,641316,2,2,1,22939,234,70061,1,675,37013,25,13938,10018,1010
4328453,1,326023,641319,2,2,2,0,0,108884,1,695,63764,30,41766,22667,1481
4328455,1,35198,641321,2,2,3,20887,221,67276,1,674,37362,30,25280,10018,1010
4328459,1,122868,641323,2,2,4,32648,191,63193,2,671,60433,6,39588,10018,1010


# Train

In [128]:
X = training.drop(columns=['song_id'])
y = training.song_id
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)

AttributeError: ignored

In [ ]:
params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting': 'gbdt',
        'learning_rate': 0.3 ,
        'verbose': 0,
        'num_leaves': 108,
        'bagging_fraction': 0.95,
        'bagging_freq': 1,
        'bagging_seed': 1,
        'feature_fraction': 0.9
        'feature_fraction_seed': 1,
        'max_bin': 256,
        'max_depth': 10,
        'num_rounds': 200,
        'metric' : 'auc'
    }

%time model_f1 = lgbm.train(params, train_set=train_set,  valid_sets=train_set, verbose_eval=5)